In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import emoji

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier


import re
import emoji
import pandas as pd
from nltk.corpus import stopwords
from wordsegment import load, segment


import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vishalthehuman/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [29]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import StackingClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np

In [30]:
# ---------------------------
# 1. Load and Combine Datasets
# ---------------------------
train_df = pd.read_csv("original/train.csv")
dev_df = pd.read_csv("original/dev.csv")
df = pd.concat([train_df, dev_df], ignore_index=True)
df

,id,text,label
0,3268,Indha ariya kandupidippin moolam neenga solla ...,0
1,6239,@vijayakumarp7959 unmai therincha nee pesu,0
2,5859,Inga erukka yella dev... boys vadakkan vadakk...,1
3,3519,பீகாரி பிரசாந்த் கிஷோரிடம் கொடுத்த 350 கோடியை ...,1
4,5136,Mumbai Bangalore la 80% percentage outsiders,1
...,...,...,...
6294,5860,This is done by those who want chaos in Tamil ...,0
6295,1441,"வடமாநிலத்தவருக்கு வேலை வாய்ப்பு கொடுப்பதை ,பணி...",0
6296,3253,நம்ம மேல எந்த தப்பும் இல்லை.... ஏன் என்றால் நா...,0
6297,4295,தம்பி உத்திரபிரதேசம் டெல்லி எல்லையில் உள்ள 90%...,1


In [31]:
df["label"].value_counts()

label
0    3900
1    2399
Name: count, dtype: int64

In [32]:
# -------------------------------
# Text Preprocessing
# -------------------------------
# Function to clean each text
# Load word segmentation model
load()

# Load stopwords
tamil_stopwords = set(stopwords.words('tamil'))
english_stopwords = set(stopwords.words('english'))

# -------------------------------
# Text Preprocessing
# -------------------------------

def preprocess_text(text):
    # Extract hashtags
    hashtags = re.findall(r"#\w+", text)

    # Process hashtags (convert to words)
    processed_hashtags = []
    for tag in hashtags:
        tag = tag.lstrip("#")  # Remove #
        words = segment(tag)   # Split compound words
        processed_hashtags.append(" ".join(words))  # Convert to sentence-like format

    # Remove hashtags from main text
    text = re.sub(r"#\w+", "", text)

    # Replace mentions with a placeholder
    text = re.sub(r'@\w+', '<USER>', text)
    
    # Convert emojis to text
    text = emoji.demojize(text, delimiters=(" ", " "))  # Adds spaces around emoji descriptions
    
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    # Merge cleaned text with processed hashtags
    final_text = text + " " + " ".join(processed_hashtags)
    final_text = final_text.replace("-", " ")  # Remove double spaces
    return final_text

# Function to remove stopwords
def remove_stopwords(text):
    words = text.split()
    filtered = [word for word in words if word.lower() not in english_stopwords and word.lower() not in tamil_stopwords]
    return ' '.join(filtered)

# Apply preprocessing
df['clean_text'] = df['text'].apply(preprocess_text).apply(remove_stopwords)

# Show sample processed data
df.sample(50)


,id,text,label,clean_text
3413,6162,​ @srikanth3505 யோவ் உன் கதையை பேசு. ஏன் சர்ச...,0,​ <USER> யோவ் கதையை பேசு. சர்ச் கிட்ட முட்டுற?...
3154,1621,இப்பதான் நான் பச்சை தமிழர்களைப்பார்க்கிறேன் .வ...,0,இப்பதான் பச்சை தமிழர்களைப்பார்க்கிறேன் .வாழத்த...
4593,3227,Dei avanunga Inga vandhu namma makkaluku Vela ...,0,Dei avanunga Inga vandhu namma makkaluku Vela ...
5065,4939,"@gowthamvasu1630 bro ,avanga epadi contact p...",0,"<USER> bro ,avanga epadi contact panrathu ,eng..."
1628,4510,உட்கார இடம் கேட்டான் 🙁🙂கொடுத்தேன் வந்தாரை வாழ...,0,உட்கார கேட்டான் slightly_frowning_face slightl...
4001,4056,இதை சொல்லீட்டே பெருமைபீத்தீட்டு போக வேண்டியதுத...,1,சொல்லீட்டே பெருமைபீத்தீட்டு போக வேண்டியதுதான்....
5565,384,ஒருநாளைக்கு ஒருநாள் வேரறுக்க தயாராவோம்,0,ஒருநாளைக்கு ஒருநாள் வேரறுக்க தயாராவோம்
473,5558,Poi sompu thukkunga da sunni media pundainga 😡...,1,Poi sompu thukkunga da sunni media pundainga e...
3470,5823,Aade muttale holi pandigai mudinthathum varuvanga,0,Aade muttale holi pandigai mudinthathum varuvanga
925,2590,வடக்கன்கள் உங்கள் மாநிலங்களில் வேலை பார்த்துக்...,1,வடக்கன்கள் உங்கள் மாநிலங்களில் வேலை பார்த்துக்...


In [33]:
df.shape

(6299, 4)

In [34]:
X = df['clean_text']

vectorizer = TfidfVectorizer(
    max_features=7000,
    ngram_range=(1,2),
    analyzer='word',
    token_pattern=r"(?u)\b\w+\b"
)

X_vectorized = vectorizer.fit_transform(X)
X_vectorized = X_vectorized.toarray()
X_vectorized.shape

(6299, 7000)

In [35]:
df.shape

(6299, 4)

In [36]:
y = df['label']
y

0       0
1       0
2       1
3       1
4       1
       ..
6294    0
6295    0
6296    0
6297    1
6298    0
Name: label, Length: 6299, dtype: int64

In [37]:
model = RandomForestClassifier(class_weight='balanced', n_estimators = 200)

model.fit(X_vectorized, y)

RandomForestClassifier(class_weight='balanced', n_estimators=200)

In [38]:
import pandas as pd

test_df = pd.read_csv("original/test.csv")
test_df

,id,text
0,4594,Humanity paatha intha slag countries prblm var...
1,3593,Avanga Holi kondada oruku poranga da
2,2533,Avangala avanga ooruke anupura vazhiya parunga.
3,6975,பைத்தியமாடா நீ.. நான் சோழிய வேளாளர். நான் என் ...
4,6011,Caste is there parayan think to marry higheru...
...,...,...
1571,6360,Ivlo varalaru pesarelada kena...aprm ethuku da...
1572,6208,@dineshkumar6695 தம்பி. யார் பேச்சையும் கேட...
1573,6871,பார்ப்பானும் பறையனும் ஒன்னு பழமொழி உள்ளது 😁😂
1574,442,நம்ம இடத்தை காப்பாத்திக்கிரது பேரு ரவுடிசம்னா ...


In [39]:
test_df['clean_text'] = test_df['text'].apply(preprocess_text).apply(remove_stopwords)
test_df.sample(50)

,id,text,clean_text
128,410,தமிழர்கள் மரியாதை கொடுப்பதால். இவ்வளவு வடவர்கள...,தமிழர்கள் மரியாதை கொடுப்பதால். இவ்வளவு வடவர்கள...
697,6855,Onnai othu puduvaargal ipdi enagaiyum sollitu ...,Onnai othu puduvaargal ipdi enagaiyum sollitu ...
1065,1952,North india la irunthu tamilnadu ku vantha fir...,North india la irunthu tamilnadu ku vantha fir...
784,2914,Nalla irukenu soluve.. yenda avanunga inga van...,Nalla irukenu soluve.. yenda avanunga inga van...
1561,812,தமிழர்களை அழிக்க திராவிட முதளாளிகள்தான் வட இந்...,தமிழர்களை அழிக்க திராவிட முதளாளிகள்தான் வட இந்...
951,3230,@manirathnam721 12மணி நேர வேலைக்கு 300 மிக க...,<USER> 12மணி நேர வேலைக்கு 300 குறைவு சார் 500ஆ...
649,3120,அவங்களுக்கு எதுக்கு இப்ப சொம்பு அடிக்குறிங்க????,அவங்களுக்கு எதுக்கு இப்ப சொம்பு அடிக்குறிங்க????
1390,1271,அவர்கள் வட இந்தியர்கள் கிடையாது தமிழர்கள் தமிழ...,வட இந்தியர்கள் கிடையாது தமிழர்கள் தமிழ்நாட்டில...
1351,4965,Karikd avaga solratu karikd rompa nallavaga av...,Karikd avaga solratu karikd rompa nallavaga av...
476,4251,மார்வாடிகளும் பீகாரிகளும் தமிழகத்தை வெகு வேகமா...,மார்வாடிகளும் பீகாரிகளும் தமிழகத்தை வெகு வேகமா...


In [40]:
X = test_df['clean_text']

vectorizer = TfidfVectorizer(
    max_features=7000,
    ngram_range=(1,2),
    analyzer='word',
    token_pattern=r"(?u)\b\w+\b"
)

X_vectorized = vectorizer.fit_transform(X)
X_vectorized = X_vectorized.toarray()
X_vectorized.shape

(1576, 7000)

In [41]:
predictions = model.predict(X_vectorized)
predictions

array([0, 1, 1, ..., 1, 1, 1])

In [42]:
test_df['predictions'] = predictions

In [43]:
test_df.drop(columns=['clean_text'], inplace=True)
test_df.drop(columns=['text'], inplace=True)

In [44]:
test_df["predictions"].value_counts()  

predictions
0    796
1    780
Name: count, dtype: int64

In [45]:
test_df.to_csv("tfidf_7000.csv", index=False, header=False)